<a href="https://colab.research.google.com/github/ifwii/MultiCNN-PowerUtilization/blob/main/Final_done_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn import metrics
import zipfile
from sklearn.preprocessing import LabelEncoder
WORD_EMBEDDING_SIZE = 50
NEWS_FT = 'news_words'
import csv
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Dropout, Activation, Embedding, Bidirectional,Conv1D,MaxPool1D,MaxPooling1D,GlobalMaxPooling1D
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from mlxtend.plotting import plot_confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
from nltk.stem import PorterStemmer
import re
nltk.download('punkt')

!pip install prettytable
from prettytable import PrettyTable


import time

In [ ]:

from sklearn.feature_extraction.text import CountVectorizer
Registers = []
Categorys = []

with open(r"C:example_1.csv", 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        Categorys.append(row[1])
        Register = row[0]
        for word in STOPWORDS:
            token = ' ' + word + ' '
            Register = Register.replace(token, ' ')
            Register = Register.replace(' ', ' ')
            Register = Register.replace('_',' ')
            Register = Register.replace('.',' ')
        Registers.append(Register)
print(len(Registers))
print(len(Categorys))
print(Registers)
vectorizer = CountVectorizer()

# Fit the vectorizer on the register data
vectorizer.fit(Registers)

# Get the vocabulary count
vocabulary_count = len(vectorizer.vocabulary_)

print("Vocabulary count:", vocabulary_count)

In [ ]:
raw_df=pd.DataFrame({"Register":Registers,"Category":Categorys})
raw_df

In [ ]:
raw_df.info()

In [ ]:
raw_df.Category.unique(),print("\n Total number of Unique Target Classes : ",raw_df.Category.nunique())


In [ ]:
stemmer = PorterStemmer()

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
REMOVE_NUM = re.compile('[\d+]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
    text: a string
    return: modified initial string
    """
    # lowercase text
    text = text.lower() 

    # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) 
    
    # Remove the XXXX values
#     text = text.replace('x', '') 
    
    # Remove white space
    text = REMOVE_NUM.sub('', text)

    #  delete symbols which are in BAD_SYMBOLS_RE from text
    text = BAD_SYMBOLS_RE.sub('', text) 

    # delete stopwords from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) 
    
    # removes any words composed of less than 2 or more than 21 letters
    text = ' '.join(word for word in text.split() if (len(word) >= 2 and len(word) <= 21))

    # Stemming the words
#     text = ' '.join([stemmer.stem(word) for word in text.split()])
    
    return text

In [ ]:
dataset=raw_df

In [ ]:
dataset['Register']=dataset['Register'].apply(clean_text)
dataset['Register']

In [ ]:
Register = dataset['Register'].values
Category = dataset['Category'].values
# print(labels)
X_train,X_test,y_train,y_test = train_test_split(Register,Category, test_size = 0.20, random_state = 42)
print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)


In [ ]:
X_train=X_train.reshape(-1,1)
X_test=X_test.reshape(-1,1)
y_train= y_train.reshape(-1,1)
y_test=y_test.reshape(-1,1)

print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)
# print(y_test)
print(y_train)

In [ ]:
average_title_size = int(sum([len(c) for c in dataset.Register])/dataset.shape[0])
print(average_title_size)

In [ ]:
#Let's Fix Some Common Parameters :
# The maximum number of words to be used. (most frequent)
#vocab_size = 50000

# Dimension of the dense embedding.
embedding_dim = 128

# Max number of words in each complaint.
#max_length = 200

# Truncate and padding options
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'

In [ ]:
# Tokenising , Converting Text To Sequences and Padding Our Data : 

#tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train.flatten())
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)
max_length = max([len(text) for text in X_train])

print(max_length)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
word_index

In [ ]:
# Converting into Text to sequences and padding :
train_seq = tokenizer.texts_to_sequences(X_train.flatten())
train_padded = pad_sequences(train_seq, maxlen=average_title_size, padding=padding_type, truncating=trunc_type)

validation_seq = tokenizer.texts_to_sequences(X_test.flatten())
validation_padded = pad_sequences(validation_seq, maxlen=average_title_size, padding=padding_type, truncating=trunc_type)

In [ ]:
X_train[3],train_seq[3],train_padded[3]
X_test[3],validation_seq[3],validation_padded[3]

In [ ]:
print('Shape of data tensor:', train_padded.shape)
print('Shape of data tensor:', validation_padded.shape)

In [ ]:
y_train.shape

In [ ]:
encode = OneHotEncoder()
training_labels = encode.fit_transform(y_train)

validation_labels = encode.transform(y_test)

In [ ]:
print(validation_labels)

In [ ]:
print(train_padded.shape)
print(validation_labels.shape)
print(validation_padded.shape)
print(training_labels.shape)
print(type(train_padded))
print(type(validation_padded))
print(type(training_labels))
print(type(validation_labels))



# The labels must be converted to arrays
# Convert the labels to arrays
training_labels = training_labels.toarray()
validation_labels = validation_labels.toarray()

print(type(training_labels))
print(type(validation_labels))

In [ ]:
def plot_graphs(history):
  plt.title('Loss VS Accuracy')
  plt.plot(history.history['loss'], label='train loss')
  plt.plot(history.history['val_loss'], label='test loss')
  plt.legend()
  
  plt.plot(history.history['accuracy'], label='train accuracy')
  plt.plot(history.history['val_accuracy'], label='test accuracy')
  plt.legend()
  plt.savefig(r"C:\my_line1_chart.png")  
  plt.show();
  return None
def loss_graph(history):
  plt.title('Loss')
  plt.plot(history.history['loss'], label='train')
  plt.plot(history.history['val_loss'], label='test')
  plt.legend()
  plt.savefig(r"C:\my_line2_chart.png")  
  plt.show();
  
  return None

def acc_graph(history):
  plt.title('Accuracy')
  plt.plot(history.history['accuracy'], label='train')
  plt.plot(history.history['val_accuracy'], label='test')
  plt.legend()
  plt.savefig(r"C:\my_line3_chart.png")  
  plt.show();

  return None


def evaluate_preds(y_true, y_preds):
    """
    Performs evaluation comparison on y_true labels vs. y_pred labels
    on a classification.
    """
    accuracy = accuracy_score(y_true, y_preds)
    precision = precision_score(y_true, y_preds, average='micro')
    recall = recall_score(y_true, y_preds, average='micro')
    f1 = f1_score(y_true, y_preds, average='micro')
    metric_dict = {"accuracy": round(accuracy, 2),
                   "precision": round(precision, 2),
                   "recall": round(recall, 2),
                   "f1": round(f1, 2)}
    print(f"Acc: {accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 score: {f1:.2f}")
    
    return metric_dict

def model_eval(model,a,b,c,d):
  score=model.evaluate(a,b,verbose=0)
  score1=model.evaluate(c,d,verbose=0)
  print("\n")
  print("Model Loss on training data ",score[0])
  print("Model Accuracy on training data: ",score[1])
  print("Model Loss on validation data",score1[0])
  print("Model Accuracy on validation data: ",score1[1])
  print("\n")
  return score,score1

def model_eval(model,a,b,c,d):
  score=model.evaluate(a,b,verbose=0)
  score1=model.evaluate(c,d,verbose=0)
  print("\n")
  print("Model Loss on training data ",score[0])
  print("Model Accuracy on training data: ",score[1])
  print("Model Loss on validation data",score1[0])
  print("Model Accuracy on validation data: ",score1[1])
  print("\n")
  return score,score1


In [ ]:
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
tf.keras.backend.clear_session()
model = Sequential()

model.add(Embedding(vocab_size,embedding_dim,input_length=train_padded.shape[1]))
model.add(Conv1D(filters=100, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(embedding_dim)))
model.add(Dense(128, activation='relu'))
model.add(Dense(41, activation='softmax'))
model.summary()
optim=tf.keras.optimizers.SGD(
    learning_rate=0.05, momentum=0.5, nesterov=True, name="SGD"
)
model.compile(
    loss='categorical_crossentropy',
    optimizer=optim,
    metrics=['accuracy'],
)

epochs = 50
batch_size = 32
filepath="weights_best_bi_lstms_n_cnn.h5"

checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max',save_weights_only=True)
start = time.perf_counter()
callbacks_list = [checkpoint,EarlyStopping(monitor='val_accuracy', mode='max', patience=10, verbose=1)]


history = model.fit(train_padded, training_labels, shuffle=True ,
                    epochs=epochs, batch_size=batch_size, 
                    callbacks=callbacks_list,validation_data=(validation_padded, validation_labels))
t23 = time.perf_counter() - start
print('Total time took for training %.3f seconds.' % t23)
hist23=history

plot_graphs(hist23),loss_graph(hist23),acc_graph(hist23)

In [ ]:
predicted = model.predict(validation_padded)
evaluate_preds(np.argmax(validation_labels, axis=1),np.argmax(predicted, axis=1))

In [ ]:
model.save("model_english.h5")


In [ ]:
Registers = []
Categorys = []

with open(r"C:\Test_data.csv", 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        Categorys.append(row[1])
        Register = row[0]
        for word in STOPWORDS:
            token = ' ' + word + ' '
            Register = Register.replace(token, ' ')
            Register = Register.replace(' ', ' ')
            Register = Register.replace('_',' ')
            Register = Register.replace('.',' ')
        Registers.append(Register)
correct_predictions = 0
total_predictions = len(Registers)
for i in range(len(Registers)):
    new_text = [clean_text(Registers[i])]
    print(new_text)
    seq = tokenizer.texts_to_sequences(new_text)
    padded = pad_sequences(seq, maxlen=average_title_size, padding=padding_type, truncating=trunc_type)
    pred = model.predict(padded)
    
    predicted_label = encode.inverse_transform(pred)
    
    predicted_category_id = np.argmax(pred)
    
    actual_category_id = Categorys[i] 
    
    if predicted_label == actual_category_id:
        correct_predictions += 1
    
    #acc = np.argmax(pred,axis=1)
    #predicted_label = encode.inverse_transform(pred)
    #print(predicted_label)
    #print(f'Product category id: {np.argmax(pred[0])}')
    #print(f'Predicted label is: {predicted_label[0]}')
    
    print(f'Product category id: {actual_category_id}')
    print(f'Predicted label is: {predicted_label[0]}')
    
accuracy = (correct_predictions / total_predictions) * 100
    
#print(f'Accuracy score: { acc.max() * 100}')
print(f'Accuracy score: {accuracy}%')


In [ ]:
Reg = input("Enter the register name: ")
token = ' ' +Reg + ' '
Reg = Reg.replace(token, ' ')
Reg = Reg.replace(' ', ' ')
Reg = Reg.replace('_',' ')
Reg = Reg.replace('.',' ')

new_text = [clean_text(Reg)]
seq = tokenizer.texts_to_sequences(new_text)
padded = pad_sequences(seq, maxlen=average_title_size, padding=padding_type, truncating=trunc_type)
pred = model.predict(padded)
acc = np.argmax(pred,axis=1)
predicted_label = encode.inverse_transform(pred)
print(f'Predicted label is: {predicted_label[0]}')
print(f'Accuracy score: {acc.max() * 100}')